# Segmenting and Clustering Neighborhoods in Toronto

## PART 1

### Importing libraries

In [61]:
import pandas as pd # library for data analsysis

### Scraping the Wikipedia website with pandas

In [62]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_wiki = pd.read_html(url)
df = df_wiki[0]
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


### Ignoring cells that Borough is Not assigned

In [63]:
df = df[df['Borough'] != 'Not assigned']
df = df.reset_index(drop=True)
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Assuring that the number of the dataframe rows is equal to the number of unique Postal Codes

In [64]:
print('Number of lines: {}'.format(df.shape[0]))
print('Number of unique postal codes: {}'.format(df['Postal Code'].nunique()))

Number of lines: 103
Number of unique postal codes: 103


### If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.

In [65]:
print('Number of lines with not assigned neighborhoods: {}'.format(df['Postal Code'][df['Neighbourhood'] == 'Not assigned'].count()))

Number of lines with not assigned neighborhoods: 0


As the number of Not assigned Neighbourhood is zero, so this dataframe is meeting the requirement.

### Number of rows of the dataframe

In [66]:
df.shape

(103, 3)

(final of PART 1)

## PART 2